# SELECCION DE ATRIBUTOS

## METODO DEL FILTRO

In [46]:
import pandas as pd
import csv
import os
my_path = os.path.abspath('')
my_path = my_path.split('\\')
my_path_py = "\\".join(my_path[:-1])

features = pd.read_csv(my_path_py+'\\2_FeatureBased\\MatrizFeatures\\18Features_Kats.csv', sep=',')

In [47]:
#features = features.iloc[:, 1:]
listadistrito = features['Dep-Prov-Distrito'].values
features = features.drop('Dep-Prov-Distrito', axis=1)
features

,Mean,Var,aCF1,Trend,Linearity,Curvature,Season,Peak,Trough,Entropy,Lumpiness,Spikiness,Lshift,Vchange,Fspots,Cpoints,KlScore,ChangeIdx
0,1.416142,28.037741,-0.019594,0.478521,0.057369,16.105723,0.259515,4.000000e+00,6.000000e+00,0.877947,1.123065e+03,0.131363,2.235336,453.036663,15,8.0,722.122511,139.0
1,7.968925,162.793719,-0.019594,0.698416,0.005168,24.055788,0.408785,3.000000e+00,1.000000e-11,0.809780,2.782585e+04,0.171691,3.804760,1683.966820,29,35.0,762.034875,101.0
2,0.896803,16.483865,-0.019594,0.511541,0.000878,-1.342372,0.448205,4.000000e+00,1.000000e-11,0.853613,4.816986e+02,0.024675,1.563966,244.599033,15,3.0,722.257988,3.0
3,1.277973,84.933434,-0.019594,0.308881,0.013646,-1.388677,0.487622,1.000000e-11,1.000000e+00,0.816967,1.288353e+04,3.156452,3.415301,1166.427782,10,1.0,185.725861,48.0
4,0.691190,6.586993,-0.019594,0.775271,0.109978,13.397455,0.358239,3.000000e+00,1.000000e-11,0.793282,1.484992e+02,0.000393,0.995897,109.760378,26,9.0,677.073513,264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,6.635039,321.295085,-0.019594,0.723762,0.005861,23.381332,0.341699,3.000000e+00,6.000000e+00,0.773567,1.297361e+05,1.234726,5.865676,3440.615516,26,8.0,803.206199,46.0
501,6.295408,399.615990,-0.019594,0.780696,0.019926,-36.253978,0.306853,2.000000e+00,5.000000e+00,0.765285,5.349600e+05,3.651875,10.209300,9195.444576,31,6.0,1820.287731,212.0
502,12.949082,3202.673627,-0.019594,0.690501,0.045318,-175.632538,0.530664,1.000000e-11,5.000000e+00,0.859131,3.691856e+07,872.908533,31.860500,83632.763411,21,8.0,4652.544429,42.0
503,6.487195,593.850800,-0.019594,0.839640,0.059928,111.830241,0.290624,1.000000e-11,1.000000e+00,0.733766,1.151792e+06,2.262564,10.373444,5509.147684,31,7.0,1597.619581,305.0


In [48]:
print("Las dimensiones de nuestros datos son: ", features.shape)

Las dimensiones de nuestros datos son:  (505, 18)


# Eliminar features varianza casi nula

In [49]:
from sklearn.feature_selection import VarianceThreshold

#sel = VarianceThreshold(threshold = 0.3)
#f_selection = sel.fit_transform(features)

sel = VarianceThreshold(threshold = 0.3)

# Fit
_ = sel.fit(features)

# Get the boolean mask
mask = sel.get_support()

features_reduced = features.loc[:, mask]


In [50]:
print("Las dimensiones de nuestros datos seleccionados son: ", features_reduced.shape)

Las dimensiones de nuestros datos seleccionados son:  (505, 13)


In [51]:
df = features_reduced.copy()

# Eliminar features con alta correlación

In [59]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_scores = [variance_inflation_factor(df.values, feature) for feature in range(len(df.columns))]
print("Por convención valores mayores a 10 se consideran redundantes")
print(" ")
i=0
listafiltrada = []
listaeliminados = []
for c in df.columns:
    print(c, "tiene ", vif_scores[i])
    if (vif_scores[i]<30):
        listafiltrada.append(c)
    else:
        listaeliminados.append(c)
    i = i +1

Por convención valores mayores a 10 se consideran redundantes
 
Mean tiene  24.050179079785458
Var tiene  210.76089029539582
Curvature tiene  3.304225302331793
Peak tiene  1.709436727762797
Trough tiene  1.9997282844043356
Lumpiness tiene  202.94057822703883
Spikiness tiene  70.72951721607414
Lshift tiene  25.98611440314274
Vchange tiene  15.742873437990614
Fspots tiene  8.463429048566388
Cpoints tiene  2.9145871526096125
KlScore tiene  6.659861938208996
ChangeIdx tiene  3.605480211746351


In [60]:
with open('FeaturesUnsupervised.csv', 'w') as f:
    write = csv.writer(f)  
    write.writerow(listafiltrada)

In [61]:
listaeliminados

['Var', 'Lumpiness', 'Spikiness']

## Cálculo de distancia

In [62]:
import math
from math import sqrt, log, floor
from sklearn.metrics import mean_squared_error
from statistics import mean
from fastdtw import fastdtw
from scipy import stats
from scipy.spatial.distance import pdist
 
#Euclidean
def euclidean(x, y):
    r=np.linalg.norm(x-y)
    if math.isnan(r):
        r=1
    #print(r)
    return r

#Fast Dynamic time warping
def fast_DTW(x, y):
    r, _ = fastdtw(x, y, dist=euclidean)
    if math.isnan(r):
        r=1
    #print(r)
    return r

#Spearman
def scorr(x, y):
    r = stats.spearmanr(x, y)[0]
    if math.isnan(r):
        r=0
    #print(r)
    return 1 - r

#RMSE
def rmse(x, y):
    r=sqrt(mean_squared_error(x,y))
    if math.isnan(r):
        r=1
    #print(r)
    return r

def lcs(a, b):  
    lengths = [[0 for j in range(len(b)+1)] for i in range(len(a)+1)]
    # row 0 and column 0 are initialized to 0 already
    for i, x in enumerate(a):
        for j, y in enumerate(b):
            if x == y:
                lengths[i+1][j+1] = lengths[i][j] + 1
            else:
                lengths[i+1][j+1] = max(lengths[i+1][j], lengths[i][j+1])
    x, y = len(a), len(b)
    result = lengths[x][y]
    return result

def discretise(x):
    return int(x * 10)

def multidim_lcs(a, b):
    a = a.applymap(discretise)
    b = b.applymap(discretise)
    rows, dims = a.shape
    lcss = [lcs(a[i+2], b[i+2]) for i in range(dims)]
    return 1 - sum(lcss) / (rows * dims)

#Correlation
def corr(x, y):
    r=np.dot(x-mean(x),y-mean(y))/((np.linalg.norm(x-mean(x)))*(np.linalg.norm(y-mean(y))))
    if math.isnan(r):
        r=0
    #print(r)
    return 1 - r

In [63]:
from sklearn.preprocessing import LabelEncoder
#from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
import scipy.cluster.hierarchy as hac
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_score, calinski_harabasz_score

DIAMETER_METHODS = ['mean_cluster', 'farthest']
CLUSTER_DISTANCE_METHODS = ['nearest', 'farthest']

def inter_cluster_distances(labels, distances, method='nearest'):
    """Calculates the distances between the two nearest points of each cluster.
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param method: `nearest` for the distances between the two nearest points in each cluster, or `farthest`
    """
    if method not in CLUSTER_DISTANCE_METHODS:
        raise ValueError(
            'method must be one of {}'.format(CLUSTER_DISTANCE_METHODS))

    if method == 'nearest':
        return __cluster_distances_by_points(labels, distances)
    elif method == 'farthest':
        return __cluster_distances_by_points(labels, distances, farthest=True)


def __cluster_distances_by_points(labels, distances, farthest=False):
    n_unique_labels = len(np.unique(labels))
    cluster_distances = np.full((n_unique_labels, n_unique_labels),
                                float('inf') if not farthest else 0)

    np.fill_diagonal(cluster_distances, 0)

    for i in np.arange(0, len(labels) - 1):
        for ii in np.arange(i, len(labels)):
            if labels[i] != labels[ii] and (
                (not farthest and
                 distances[i, ii] < cluster_distances[labels[i], labels[ii]])
                    or
                (farthest and
                 distances[i, ii] > cluster_distances[labels[i], labels[ii]])):
                cluster_distances[labels[i], labels[ii]] = cluster_distances[
                    labels[ii], labels[i]] = distances[i, ii]
    return cluster_distances


def diameter(labels, distances, method='farthest'):
    """Calculates cluster diameters
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param method: either `mean_cluster` for the mean distance between all elements in each cluster, or `farthest` for the distance between the two points furthest from each other
    """
    if method not in DIAMETER_METHODS:
        raise ValueError('method must be one of {}'.format(DIAMETER_METHODS))

    n_clusters = len(np.unique(labels))
    diameters = np.zeros(n_clusters)

    if method == 'mean_cluster':
        for i in range(0, len(labels) - 1):
            for ii in range(i + 1, len(labels)):
                if labels[i] == labels[ii]:
                    diameters[labels[i]] += distances[i, ii]

        for i in range(len(diameters)):
            diameters[i] /= sum(labels == i)

    elif method == 'farthest':
        for i in range(0, len(labels) - 1):
            for ii in range(i + 1, len(labels)):
                if labels[i] == labels[ii] and distances[i, ii] > diameters[
                        labels[i]]:
                    diameters[labels[i]] = distances[i, ii]
    return diameters

def dunn(labels, distances, diameter_method='farthest',
         cdist_method='nearest'):
    """
    Dunn index for cluster validation (larger is better).
    
    .. math:: D = \\min_{i = 1 \\ldots n_c; j = i + 1\ldots n_c} \\left\\lbrace \\frac{d \\left( c_i,c_j \\right)}{\\max_{k = 1 \\ldots n_c} \\left(diam \\left(c_k \\right) \\right)} \\right\\rbrace
    
    where :math:`d(c_i,c_j)` represents the distance between
    clusters :math:`c_i` and :math:`c_j`, and :math:`diam(c_k)` is the diameter of cluster :math:`c_k`.
    Inter-cluster distance can be defined in many ways, such as the distance between cluster centroids or between their closest elements. Cluster diameter can be defined as the mean distance between all elements in the cluster, between all elements to the cluster centroid, or as the distance between the two furthest elements.
    The higher the value of the resulting Dunn index, the better the clustering
    result is considered, since higher values indicate that clusters are
    compact (small :math:`diam(c_k)`) and far apart (large :math:`d \\left( c_i,c_j \\right)`).
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param diameter_method: see :py:function:`diameter` `method` parameter
    :param cdist_method: see :py:function:`diameter` `method` parameter
    
    .. [Kovacs2005] Kovács, F., Legány, C., & Babos, A. (2005). Cluster validity measurement techniques. 6th International Symposium of Hungarian Researchers on Computational Intelligence.
    """

    labels = LabelEncoder().fit(labels).transform(labels)
    
    

    ic_distances = inter_cluster_distances(labels, distances, cdist_method)
    #print("IC",ic_distances)
    if len(ic_distances[ic_distances.nonzero()])==0:
        min_distance = 0
    else:
        min_distance = min(ic_distances[ic_distances.nonzero()])
    max_diameter = max(diameter(labels, distances, diameter_method))
    
    

    return min_distance / max_diameter

In [64]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN


# Definición de número de grupos

In [65]:
k=6

# CÁLCULO DE MATRICES DE DISTANCIA

In [66]:
df

,Mean,Var,Curvature,Peak,Trough,Lumpiness,Spikiness,Lshift,Vchange,Fspots,Cpoints,KlScore,ChangeIdx
0,1.416142,28.037741,16.105723,4.000000e+00,6.000000e+00,1.123065e+03,0.131363,2.235336,453.036663,15,8.0,722.122511,139.0
1,7.968925,162.793719,24.055788,3.000000e+00,1.000000e-11,2.782585e+04,0.171691,3.804760,1683.966820,29,35.0,762.034875,101.0
2,0.896803,16.483865,-1.342372,4.000000e+00,1.000000e-11,4.816986e+02,0.024675,1.563966,244.599033,15,3.0,722.257988,3.0
3,1.277973,84.933434,-1.388677,1.000000e-11,1.000000e+00,1.288353e+04,3.156452,3.415301,1166.427782,10,1.0,185.725861,48.0
4,0.691190,6.586993,13.397455,3.000000e+00,1.000000e-11,1.484992e+02,0.000393,0.995897,109.760378,26,9.0,677.073513,264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,6.635039,321.295085,23.381332,3.000000e+00,6.000000e+00,1.297361e+05,1.234726,5.865676,3440.615516,26,8.0,803.206199,46.0
501,6.295408,399.615990,-36.253978,2.000000e+00,5.000000e+00,5.349600e+05,3.651875,10.209300,9195.444576,31,6.0,1820.287731,212.0
502,12.949082,3202.673627,-175.632538,1.000000e-11,5.000000e+00,3.691856e+07,872.908533,31.860500,83632.763411,21,8.0,4652.544429,42.0
503,6.487195,593.850800,111.830241,1.000000e-11,1.000000e+00,1.151792e+06,2.262564,10.373444,5509.147684,31,7.0,1597.619581,305.0


In [67]:
df = df.drop(listaeliminados, axis=1)

In [68]:
n = df.shape[0]

### MATRIZ DTW

In [69]:
#DTW
f_dtw_dist = np.zeros((n,n))
for i in range(0,n):
    for j in range(0,n):
        f_dtw_dist[i,j] = fast_DTW(df.iloc[i].values.flatten(), df.iloc[j].values.flatten())

In [70]:
f_dtw_distDF = pd.DataFrame(f_dtw_dist)
f_dtw_distDF.to_csv(my_path_py+'\\3_FeatureSelection\\MatricesDistancia\\MatrizDTW_FBFS.csv', index=False)

### MATRIZ EUCLIDIANA

In [71]:
#Euclidean
f_euclidean_dist = np.zeros((n,n))
for i in range(0,n):
    #print("i",i)
    for j in range(1,n):
         f_euclidean_dist[i,j] = euclidean(df.iloc[i].values.flatten(), df.iloc[j].values.flatten())

In [72]:
f_euclidean_distDF = pd.DataFrame(f_euclidean_dist)
f_euclidean_distDF.to_csv(my_path_py+'\\3_FeatureSelection\\MatricesDistancia\\MatrizEuclidiana_FBFS.csv', index=False)

### MATRIZ CORRELACIÓN PEARSON 

In [73]:
#Corr
corr_dist = np.zeros((n,n))
for i in range(0,n):
    for j in range(0,n):
            corr_dist[i,j] = corr(df.iloc[i].values.flatten(), df.iloc[j].values.flatten())

In [74]:
corr_distDF = pd.DataFrame(corr_dist)
corr_distDF.to_csv(my_path_py+'\\3_FeatureSelection\\MatricesDistancia\\MatrizPearson_FBFS.csv', index=False)

### MATRIZ CORRELACIÓN SPEARMAN

In [75]:
#scorr
f_scorr_dist = np.zeros((n,n))
for i in range(0,n):
    for j in range(0,n):
        f_scorr_dist[i,j] = scorr(df.iloc[i].values.flatten(), df.iloc[j].values.flatten())

In [76]:
f_scorr_distDF = pd.DataFrame(f_scorr_dist)
f_scorr_distDF.to_csv(my_path_py+'\\3_FeatureSelection\\MatricesDistancia\\MatrizSpearman_FBFS.csv', index=False)

# CLUSTERING

In [77]:
#Experimentos HAC
HAC_EUCLIDEAN=[]
HAC_CORRELATION=[]
HAC_SPEARMAN=[]
HAC_DTW=[]

HAC_EUCLIDEAN_CHZ=[]
HAC_CORRELATION_CHZ=[]
HAC_SPEARMAN_CHZ=[]
HAC_DTW_CHZ=[]

HAC_EUCLIDEAN_DUNN=[]
HAC_CORRELATION_DUNN=[]
HAC_SPEARMAN_DUNN=[]
HAC_DTW_DUNN=[]

HAC_EUCLIDEAN_DAVID=[]
HAC_CORRELATION_DAVID=[]
HAC_SPEARMAN_DAVID=[]
HAC_DTW_DAVID=[]

HAC_euc = AgglomerativeClustering(n_clusters=k).fit_predict(f_euclidean_dist)
print("HAC + euclidian distance: ")

sil = silhouette_score(f_euclidean_dist, HAC_euc)
chz = calinski_harabasz_score(f_euclidean_dist, HAC_euc)
try:
    dunn_= dunn(HAC_euc, f_euclidean_dist, 'farthest', 'farthest')
except:
    dunn_=np.nan
david_= davies_bouldin_score(f_euclidean_dist, HAC_euc)

print("SC: ", sil)
print("CHZ: ", chz)
print("DUNN: ", dunn_)
print("DAV-BOUD: ", david_)

HAC_EUCLIDEAN.append(sil)
HAC_EUCLIDEAN_CHZ.append(chz)
HAC_EUCLIDEAN_DUNN.append(dunn_)
HAC_EUCLIDEAN_DAVID.append(david_)

print("----------------------------------")

HAC_corr = AgglomerativeClustering(n_clusters=k).fit_predict(corr_dist)
print("HAC + corr distance: ")
sil = silhouette_score(corr_dist, HAC_corr)
chz = calinski_harabasz_score(corr_dist, HAC_corr)
try:
    dunn_ = dunn(HAC_corr, corr_dist, 'farthest', 'farthest')
except:
    dunn_=np.nan
david_= davies_bouldin_score(corr_dist, HAC_corr)

print("SC: ", sil)
print("CHZ: ", chz)
print("DUNN: ", dunn_)
print("DAV-BOUD: ", david_)

HAC_CORRELATION.append(sil)
HAC_CORRELATION_CHZ.append(chz)
HAC_CORRELATION_DUNN.append(dunn_)
HAC_CORRELATION_DAVID.append(david_)

print("----------------------------------")

HAC_scorr = AgglomerativeClustering(n_clusters=k).fit_predict(f_scorr_dist)
print("HAC + scorr distance: ")
sil = silhouette_score(f_scorr_dist, HAC_scorr)
chz = calinski_harabasz_score(f_scorr_dist, HAC_scorr)
try:
    dunn_ = dunn(HAC_scorr, f_scorr_dist, 'farthest', 'farthest')
except:
    dunn_=np.nan
david_= davies_bouldin_score(f_scorr_dist, HAC_scorr)

print("SC: ", sil)
print("CHZ: ", chz)
print("DUNN: ", dunn_)
print("DAV-BOUD: ", david_)

HAC_SPEARMAN.append(sil)
HAC_SPEARMAN_CHZ.append(chz)
HAC_SPEARMAN_DUNN.append(dunn_)
HAC_SPEARMAN_DAVID.append(david_)

print("----------------------------------")

HAC_dtw = AgglomerativeClustering(n_clusters=k).fit_predict(f_dtw_dist)
print("HAC + dtw distance: ")
sil = silhouette_score(f_dtw_dist, HAC_dtw)
chz = calinski_harabasz_score(f_dtw_dist, HAC_dtw)
try:
    dunn_ = dunn(HAC_dtw, f_dtw_dist, 'farthest', 'farthest')
except:
    dunn_=np.nan
david_= davies_bouldin_score(f_dtw_dist, HAC_dtw)

print("SC: ", sil)
print("CHZ: ", chz)
print("DUNN: ", dunn_)
print("DAV-BOUD: ", david_)

HAC_DTW.append(sil)
HAC_DTW_CHZ.append(chz)
HAC_DTW_DUNN.append(dunn_)
HAC_DTW_DAVID.append(david_)

HAC + euclidian distance: 
SC:  0.8785265047829188
CHZ:  6338.754832377213
DUNN:  1.0823177407045927
DAV-BOUD:  0.25587212362993556
----------------------------------
HAC + corr distance: 
SC:  0.5875777137491371
CHZ:  1366.4267369809547
DUNN:  0.851326622977221
DAV-BOUD:  0.5980819998302872
----------------------------------
HAC + scorr distance: 


C:\ProgramData\Anaconda3\lib\site-packages\scipy\cluster\hierarchy.py:826: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


SC:  0.39150265115246163
CHZ:  349.0975169127851
DUNN:  0.7387590196194002
DAV-BOUD:  1.020594235349151
----------------------------------
HAC + dtw distance: 
SC:  0.8740112363147372
CHZ:  6294.693930453884
DUNN:  1.048660137898133
DAV-BOUD:  0.2528272369528758


In [78]:
KM_EUCLIDEAN=[]
KM_CORRELATION=[]
KM_SPEARMAN=[]
KM_DTW=[]

KM_EUCLIDEAN_CHZ=[]
KM_CORRELATION_CHZ=[]
KM_SPEARMAN_CHZ=[]
KM_DTW_CHZ=[]

KM_EUCLIDEAN_DUNN=[]
KM_CORRELATION_DUNN=[]
KM_SPEARMAN_DUNN=[]
KM_DTW_DUNN=[]

KM_EUCLIDEAN_DAVID=[]
KM_CORRELATION_DAVID=[]
KM_SPEARMAN_DAVID=[]
KM_DTW_DAVID=[]

#Experimentos K-Means
km_euc = KMeans(n_clusters=k).fit_predict(f_euclidean_dist)
print("KM + euclidian distance: ")
sil = silhouette_score(f_euclidean_dist, km_euc)
chz = calinski_harabasz_score(f_euclidean_dist, km_euc)
try:
    dunn_ = dunn(km_euc, f_euclidean_dist, 'farthest', 'farthest')
except:
    dunn_=np.nan
david_ = davies_bouldin_score(f_euclidean_dist, km_euc)

print("SC: ", sil)
print("CHZ: ", chz)
print("DUNN: ", dunn_)
print("DAV-BOULD: ", david_)

KM_EUCLIDEAN.append(sil)
KM_EUCLIDEAN_CHZ.append(chz)
KM_EUCLIDEAN_DUNN.append(dunn_)
KM_EUCLIDEAN_DAVID.append(david_)

print("------------------------------")

km_corr = KMeans(n_clusters=k).fit_predict(corr_dist)
print("KM + corr distance: ")
sil = silhouette_score(corr_dist, km_corr)
chz = calinski_harabasz_score(corr_dist, km_corr)
try:
    dunn_ = dunn(km_corr, corr_dist, 'farthest', 'farthest') 
except:
    dunn_=np.nan
david_ = davies_bouldin_score(corr_dist, km_corr)

print("SC: ", sil)
print("CHZ: ", chz)
print("DUNN: ", dunn_)
print("DAV-BOULD: ", david_)

KM_CORRELATION.append(sil)
KM_CORRELATION_CHZ.append(chz)
KM_CORRELATION_DUNN.append(dunn_)
KM_CORRELATION_DAVID.append(david_)

print("------------------------------")

km_scorr = KMeans(n_clusters=k).fit_predict(f_scorr_dist)
print("KM + scorr distance: ")
sil = silhouette_score(f_scorr_dist, km_scorr)
chz = calinski_harabasz_score(f_scorr_dist, km_scorr)
try:
    dunn_ = dunn(km_scorr, f_scorr_dist, 'farthest', 'farthest')
except:
    dunn_=np.nan
david_ = davies_bouldin_score(f_scorr_dist, km_scorr)

print("SC: ", sil)
print("CHZ: ", chz)
print("DUNN: ", dunn_)
print("DAV-BOULD: ", david_)

KM_SPEARMAN.append(sil)
KM_SPEARMAN_CHZ.append(chz)
KM_SPEARMAN_DUNN.append(dunn_)
KM_SPEARMAN_DAVID.append(david_)

print("------------------------------")

km_dtw = KMeans(n_clusters=k).fit_predict(f_dtw_dist)
print("KM + dtw distance: ")
sil = silhouette_score(f_dtw_dist, km_dtw)
chz = calinski_harabasz_score(f_dtw_dist, km_dtw)
try:
    dunn_ = dunn(km_dtw, f_dtw_dist, 'farthest', 'farthest')
except:
    dunn_=np.nan
david_ = davies_bouldin_score(f_dtw_dist, km_dtw)

print("SC: ", sil)
print("CHZ: ", chz)
print("DUNN: ", dunn_)
print("DAV-BOULD: ", david_)

KM_DTW.append(sil)
KM_DTW_CHZ.append(chz)
KM_DTW_DUNN.append(dunn_)
KM_DTW_DAVID.append(david_)


KM + euclidian distance: 
SC:  0.8578706100444677
CHZ:  6607.269465368838
DUNN:  0.8499382828851052
DAV-BOULD:  0.30505960809159094
------------------------------
KM + corr distance: 
SC:  0.5806454194695692
CHZ:  1531.255910796374
DUNN:  0.8164281077263447
DAV-BOULD:  0.6339729635599659
------------------------------
KM + scorr distance: 
SC:  0.4219568365266967
CHZ:  380.7790756633296
DUNN:  0.7387590196194002
DAV-BOULD:  0.9666775219049598
------------------------------
KM + dtw distance: 
SC:  0.8541791803301831
CHZ:  6368.578708931667
DUNN:  0.8978544516678802
DAV-BOULD:  0.2946527109310228


In [79]:
DBSCAN_EUCLIDEAN=[]
DBSCAN_CORRELATION=[]
DBSCAN_SPEARMAN=[]
DBSCAN_DTW=[]

DBSCAN_EUCLIDEAN_CHZ=[]
DBSCAN_CORRELATION_CHZ=[]
DBSCAN_SPEARMAN_CHZ=[]
DBSCAN_DTW_CHZ=[]

DBSCAN_EUCLIDEAN_DUNN=[]
DBSCAN_CORRELATION_DUNN=[]
DBSCAN_SPEARMAN_DUNN=[]
DBSCAN_DTW_DUNN=[]

DBSCAN_EUCLIDEAN_DAVID=[]
DBSCAN_CORRELATION_DAVID=[]
DBSCAN_SPEARMAN_DAVID=[]
DBSCAN_DTW_DAVID=[]

#CON EUCLIDEAN
DB_euc = DBSCAN(eps=170, min_samples=3).fit_predict(f_euclidean_dist)
print("DBSCAN + euclidian distance: ")

sil =  silhouette_score(f_euclidean_dist, DB_euc)
CHZ_ = calinski_harabasz_score(f_euclidean_dist, DB_euc) 
try:
    dunn_ = dunn(DB_euc, f_euclidean_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
david_ = davies_bouldin_score(f_euclidean_dist, DB_euc)

print("SC: ", sil)
print("CHZ: ", CHZ_)
print("DUNN: ", dunn_)
print("DAV-BOULD: ", david_)

DBSCAN_EUCLIDEAN.append(sil)
DBSCAN_EUCLIDEAN_CHZ.append(CHZ_)
DBSCAN_EUCLIDEAN_DUNN.append(dunn_)
DBSCAN_EUCLIDEAN_DAVID.append(david_)

print("------------------------------")


#CON CORRELATION
DB_corr = DBSCAN(eps=0.45, min_samples=3).fit_predict(corr_dist)
print("DBSCAN + corr distance: ")

sil = silhouette_score(corr_dist, DB_corr)
CHZ_ = calinski_harabasz_score(corr_dist, DB_corr)
try:
    dunn_ = dunn(DB_corr, corr_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
david_ = davies_bouldin_score(corr_dist, DB_corr)

print("SC: ", sil)
print("CHZ: ", CHZ_)
print("DUNN: ", dunn_)
print("DAV-BOULD: ", david_)

DBSCAN_CORRELATION.append(sil)
DBSCAN_CORRELATION_CHZ.append(CHZ_)
DBSCAN_CORRELATION_DUNN.append(dunn_)
DBSCAN_CORRELATION_DAVID.append(david_)

print("------------------------------")

#CON SPEARMAN
DB_scorr = DBSCAN(eps=0.45, min_samples=3).fit_predict(f_scorr_dist)
print("DBSCAN + scorr distance: ")

sil = silhouette_score(f_scorr_dist, DB_scorr)
CHZ_ = calinski_harabasz_score(f_scorr_dist, DB_scorr)
try:
    dunn_ = dunn(DB_scorr, f_scorr_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
david_ = davies_bouldin_score(f_scorr_dist, DB_scorr)

print("SC: ", sil)
print("CHZ: ", CHZ_)
print("DUNN: ", dunn_)
print("DAV-BOULD: ", david_)
DBSCAN_SPEARMAN.append(sil)
DBSCAN_SPEARMAN_CHZ.append(CHZ_)
DBSCAN_SPEARMAN_DUNN.append(dunn_)
DBSCAN_SPEARMAN_DAVID.append(david_)

print("------------------------------")

#CON D TIME WARPING
DB_dtw = DBSCAN(eps=170, min_samples=3).fit_predict(f_dtw_dist)
print("DBSCAN + dtw distance: ")

sil = silhouette_score(f_dtw_dist, DB_dtw)
CHZ_ = calinski_harabasz_score(f_dtw_dist, DB_dtw)
try:
    dunn_ = dunn(DB_dtw, f_dtw_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
david_ = davies_bouldin_score(f_dtw_dist, DB_dtw)

print("SC: ", sil)
print("CHZ: ", CHZ_)
print("DUNN: ", dunn_)
print("DAV-BOULD: ", david_)
DBSCAN_DTW.append(sil)
DBSCAN_DTW_CHZ.append(CHZ_)
DBSCAN_DTW_DUNN.append(dunn_)
DBSCAN_DTW_DAVID.append(david_)

DBSCAN + euclidian distance: 
SC:  -0.5569416925728353
CHZ:  0.41189990940884863
DUNN:  2.0032627641599044e-05
DAV-BOULD:  1.6384171477273881
------------------------------
DBSCAN + corr distance: 
SC:  0.17114048500760087
CHZ:  105.01375796229746
DUNN:  0.7411983835976086
DAV-BOULD:  1.0923175997157908
------------------------------
DBSCAN + scorr distance: 
SC:  -0.09514507091259121
CHZ:  5.473580581686799
DUNN:  0.5694398864740464
DAV-BOULD:  1.165594854474498
------------------------------
DBSCAN + dtw distance: 
SC:  -0.6344809884942453
CHZ:  0.2050395000274255
DUNN:  8.641024087754016e-06
DAV-BOULD:  1.654019614814473


In [80]:
aux2 = pd.DataFrame()
aux2['Distrito'] = listadistrito
aux2['Cluster KM'] = km_euc
aux2['Cluster HAC '] = HAC_euc
aux2['Cluster DB SP'] = DB_scorr
aux2.to_csv('ClusterFBFS_2015al2020.csv')

In [81]:
Scores_ = pd.DataFrame()
Scores_['Metrica'] = ['Euclidean', 'Pearson', 'Spearman', 'DTW']

In [82]:
Scores_['SIL HAC 8'] = [ HAC_EUCLIDEAN[0], HAC_CORRELATION[0], HAC_SPEARMAN[0], HAC_DTW[0]]
Scores_['SIL KM 8'] = [ KM_EUCLIDEAN[0], KM_CORRELATION[0], KM_SPEARMAN[0], KM_DTW[0]]
Scores_['SIL DB 8'] = [ DBSCAN_EUCLIDEAN[0], DBSCAN_CORRELATION[0], DBSCAN_SPEARMAN[0], DBSCAN_DTW[0]]

Scores_['CHZ HAC 8'] = [ HAC_EUCLIDEAN_CHZ[0], HAC_CORRELATION_CHZ[0], HAC_SPEARMAN_CHZ[0], HAC_DTW_CHZ[0]]
Scores_['CHZ KM 8'] = [ KM_EUCLIDEAN_CHZ[0], KM_CORRELATION_CHZ[0], KM_SPEARMAN_CHZ[0], KM_DTW_CHZ[0]]
Scores_['CHZ DB 8'] = [ DBSCAN_EUCLIDEAN_CHZ[0], DBSCAN_CORRELATION_CHZ[0], DBSCAN_SPEARMAN_CHZ[0], DBSCAN_DTW_CHZ[0]]

Scores_['DAVIES HAC 8'] = [ HAC_EUCLIDEAN_DAVID[0], HAC_CORRELATION_DAVID[0], HAC_SPEARMAN_DAVID[0], HAC_DTW_DAVID[0]]
Scores_['DAVIES KM 8'] = [ KM_EUCLIDEAN_DAVID[0], KM_CORRELATION_DAVID[0], KM_SPEARMAN_DAVID[0], KM_DTW_DAVID[0]]
Scores_['DAVIES DB 8'] = [ DBSCAN_EUCLIDEAN_DAVID[0], DBSCAN_CORRELATION_DAVID[0], DBSCAN_SPEARMAN_DAVID[0], DBSCAN_DTW_DAVID[0]]

In [83]:
Scores_.to_csv(my_path_py+'\\3_FeatureSelection\\Resultados_Scores\\Scores8F_Based.csv')

In [84]:
Scores_

,Metrica,SIL HAC 8,SIL KM 8,SIL DB 8,CHZ HAC 8,CHZ KM 8,CHZ DB 8,DAVIES HAC 8,DAVIES KM 8,DAVIES DB 8
0,Euclidean,0.878527,0.857871,-0.556942,6338.754832,6607.269465,0.411900,0.255872,0.305060,1.638417
1,Pearson,0.587578,0.580645,0.171140,1366.426737,1531.255911,105.013758,0.598082,0.633973,1.092318
2,Spearman,0.391503,0.421957,-0.095145,349.097517,380.779076,5.473581,1.020594,0.966678,1.165595
3,DTW,0.874011,0.854179,-0.634481,6294.693930,6368.578709,0.205040,0.252827,0.294653,1.654020
